In [1]:
import pandas as pd
import os
import numpy as np
from datetime import datetime

Test with poland

In [2]:
#file_name='minimal_seats-Poland_without91.xlsx'

In [3]:
file_name='minimal_seats-Australia.xlsx'

In [4]:

def country_df(file_name): 
    df=pd.read_excel(f'mvws/{file_name}')
    ###get country name part###
    first_split=file_name.split('-')
    country_name=first_split[1].split('.')[0]
    ### get dates from excel###
    dates=df['Key_1'].unique()
    dates_dt_1= pd.to_datetime(dates,errors='coerce',format='-%b-%y').sort_values()
    dates_dt_2= pd.to_datetime(dates,errors='coerce',format='%Y-%m').sort_values() #sometimes date format changes
    new_df=pd.DataFrame()
    if not dates_dt_2.empty:       ## if date format changed we need this convoluted bit here:
        dates_dt_2_1=dates_dt_2.strftime('%Y-%m')
        dates_dt_2_formatted=dates_dt_2.strftime('-%b-%y')

        sort_dates_str_1=dates_dt_1.strftime('-%b-%y')

        for i,date in enumerate(dates_dt_2_1):
            relevant_rows=df[df['Key_1']==date]
            parties=relevant_rows['Key_2'].tolist()
            mvws=relevant_rows['Value_1'].tolist()
            new_df[i]=pd.Series([parties,mvws])
            new_df.rename(columns={i:dates_dt_2_formatted[i]},inplace=True)

    ### create df as wanted
    for date in sort_dates_str_1:
        relevant_rows=df[df['Key_1']==date]
        parties=relevant_rows['Key_2'].tolist()
        mvws=relevant_rows['Value_1'].tolist()
        new_df[date]=pd.Series([parties,mvws])
    new_df = new_df.loc[:, new_df.columns.notna()]        
    new_df=new_df.add_prefix(f'{country_name}')
    return new_df

In [5]:
#poland_df=country_df(file_name)

In [6]:
all_mvw_files=[i for i in os.listdir('mvws') if i.endswith('.xlsx')]

In [7]:
country_dfs = []
for mvw_file in all_mvw_files: 
    df=country_df(mvw_file)
    country_dfs.append(df)
all_countries_df= pd.concat(country_dfs,axis=1)

In [8]:
indexnames= ['parties','mvws']
all_countries_df.index = indexnames

Now add the original seats to the df

In [9]:
def update_prelims(file_name,all_countries_df): 
    file= pd.ExcelFile(f'prelims/{file_name}')
    ### Part for getting the original Seats
    df= file.parse('Transformed Data')
    dates=df['YearMonth'].unique()
    country_name=file_name.split('-')[1].split('.')[0]
    formatted_dates=[f"{country_name}{date}" for date in dates]
    
    for date in formatted_dates: 
        try: 
            if date in all_countries_df.columns:
                relevant_rows= df[df['YearMonth']==date.replace(f'{country_name}','')] 
                parties_from_file= relevant_rows['Party'].tolist()
                parties_from_df=all_countries_df.loc['parties',date]
                if parties_from_df==parties_from_file: 
                    if 'Seats' not in all_countries_df.index:
                        all_countries_df.loc['Seats']=[np.nan]*len(all_countries_df.columns)
                    seats= relevant_rows['# of Seats'].tolist()
                    all_countries_df.at['Seats',date]= seats
                else: all_countries_df.at['Seats', date] = 'error'
            else: raise ValueError
        except ValueError: 
            country_part=date[:-7]
            date_part=date[-7:]
            date_part_dt=datetime.strptime(date_part, '%Y-%m')
            date_part=date_part_dt.strftime('-%b-%y')
            date_correctformat=f'{country_part}{date_part}'
        
            if date_correctformat in all_countries_df.columns:
                relevant_rows= df[df['YearMonth']==date.replace(f'{country_name}','')] 
                parties_from_file= relevant_rows['Party'].tolist()
                parties_from_df=all_countries_df.loc['parties',date_correctformat]
                if parties_from_df==parties_from_file: 
                    if 'Seats' not in all_countries_df.index:
                        all_countries_df.loc['Seats']=[np.nan]*len(all_countries_df.columns)
                    seats= relevant_rows['# of Seats'].tolist()
                    all_countries_df.at['Seats',date_correctformat]= seats
                else: 
                    all_countries_df.at['Seats', date] = 'error'
            else: print(f'something is wrong with {date}')
    ### part for getting n_in_year:
    n_df=file.parse('n per Year')
    for index,row1 in n_df.iterrows(): 
        date=row1['Key']
        formatted_date=f'{country_name}{date}'
        try:
            if formatted_date in all_countries_df.columns: 
                if 'n per Year' not in all_countries_df.index: 
                    all_countries_df.loc['n per Year']=[np.nan]*len(all_countries_df.columns)
                all_countries_df.at['n per Year',formatted_date]= row1['Value_1']
            else: raise ValueError
        except ValueError: 
            date_part_dt=datetime.strptime(date, '%Y-%m')
            date_part=date_part_dt.strftime('-%b-%y')
            date_correctformat=f'{country_name}{date_part}'
            if date_correctformat in all_countries_df.columns: 
                if 'n per Year' not in all_countries_df.index: 
                    all_countries_df.loc['n per Year']=[np.nan]*len(all_countries_df.columns)
                all_countries_df.at['n per Year',date_correctformat]= row1['Value_1']
    ### part for getting Total Seats: 
    Q_df=file.parse('Total Seats per Year')
    for index,row1 in Q_df.iterrows(): 
        date=row1['Key']
        formatted_date=f'{country_name}{date}'
        try:
            if formatted_date in all_countries_df.columns: 
                if 'Q' not in all_countries_df.index: 
                    all_countries_df.loc['Q']=[np.nan]*len(all_countries_df.columns)
                all_countries_df.at['Q',formatted_date]= row1['Value_1']
            else: raise ValueError
        except ValueError: 
            date_part_dt=datetime.strptime(date, '%Y-%m')
            date_part=date_part_dt.strftime('-%b-%y')
            date_correctformat=f'{country_name}{date_part}'
            if date_correctformat in all_countries_df.columns: 
                if 'Q' not in all_countries_df.index: 
                    all_countries_df.loc['Q']=[np.nan]*len(all_countries_df.columns)
                all_countries_df.at['Q',date_correctformat]= row1['Value_1']
    return all_countries_df

In [10]:
file_name='Preliminaries-Israel_99_modified.xlsx'

In [11]:
df_2=update_prelims(file_name,all_countries_df)

now add the power indices: 

In [12]:
def update_powerindices(file_name,all_countries_df): 
    file=pd.ExcelFile(f'powerindices/{file_name}')
    for sheet_name in file.sheet_names: 
        df=pd.read_excel(file,sheet_name=sheet_name)
        country_name=file_name.split('-')[1].split('.')[0]
        date=f'{country_name}{sheet_name}'
        try:
            if date in all_countries_df: 
                parties_from_sheet=df['Party'].tolist()
                parties_from_df=all_countries_df.loc['parties',date]
                if parties_from_sheet==parties_from_df: 
                    for col in df.columns[1:]: 
                        if col not in all_countries_df.index: 
                            all_countries_df.loc[col] = [np.nan] * len(all_countries_df.columns)    
                        all_countries_df.at[col, date] = df[col].tolist()
                else: 
                    all_countries_df.at[col, date] = 'error' 
                    print(parties_from_sheet)
            else: raise ValueError
        except:
            date_part_dt=datetime.strptime(sheet_name,'%Y-%m')
            date_part=date_part_dt.strftime('-%b-%y')
            date_correctformat = f'{country_name}{date_part}'
            
            if date_correctformat in all_countries_df.columns:
                parties_from_sheet=df['Party'].tolist()
                parties_from_df=all_countries_df.loc['parties',date_correctformat]
                if parties_from_sheet==parties_from_df: 
                    for col in df.columns[1:]: 
                        if col not in all_countries_df.index: 
                            all_countries_df.loc[col] = [np.nan] * len(all_countries_df.columns)    
                        all_countries_df.at[col, date_correctformat] = df[col].tolist() 
            else:  print(f'something is wrong with {date} into {date_correctformat}')
                    
                
            
    return all_countries_df

In [13]:
file_name='power_indices-Israel_99_modified.xlsx'

In [14]:
df3=update_powerindices(file_name,all_countries_df)

In [15]:
all_originalseats_files=[i for i in os.listdir('prelims') if i.endswith('.xlsx')]

In [16]:
for prelimfile in all_originalseats_files: 
    all_countries_df=update_prelims(prelimfile,all_countries_df)

In [17]:
all_powerfiles=[i for i in os.listdir('powerindices') if i.endswith('.xlsx')]

In [18]:
for powerfile in all_powerfiles: 
    all_countries_df=update_powerindices(powerfile,all_countries_df)

In [21]:
with pd.ExcelWriter('results/complete_dataframe.xlsx', engine='xlsxwriter') as writer:
    all_countries_df.to_excel(writer,index=True)